## 2.routing

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

In [4]:
try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print(f"Language model initialized: {llm.model}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None

Language model initialized: models/gemini-2.5-flash


E0000 00:00:1760418853.705584 21569429 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


1. 가상의 서브 에이전트 핸들러 정의(ADK의 sub_agents 역할)

In [5]:
def booking_handler(request: str) -> str:
    """Simulates the Booking Agent handling a request."""
    print("\n--- 예약 핸들러에게 위임 중 ---")
    return f"예약 핸들러가 요청을 처리했습니다: '{request}'. 결과: 예약 작업을 시뮬레이션했습니다."

def info_handler(request: str) -> str:
    """Simulates the Info Agent handling a request."""
    print("\n--- 정보 핸들러에게 위임 중 ---")
    return f"정보 핸들러가 요청을 처리했습니다: '{request}'. 결과: 정보 조회를 시뮬레이션했습니다."

def unclear_handler(request: str) -> str:
    """Handles requests that couldn't be delegated."""
    print("\n--- 불명확한 요청 처리 중 ---")
    return f"코디네이터가 요청을 위임할 수 없습니다: '{request}'. 내용을 구체적으로 알려주세요."

2. 라우터 체인 정의

In [6]:
coordinator_router_prompt = ChatPromptTemplate.from_messages([
    ("system", """사용자의 요청을 분석하여 어떤 전문 핸들러가 처리해야 하는지 결정하세요.
     - 요청이 항공권 또는 호텔 예약과 관련된 경우,
       'booker'를 출력하세요.
     - 그 외 일반 정보 질문의 경우 'info'를 출력하세요.
     - 요청이 불명확하거나 두 범주에 모두 해당하지 않으면,
       'unclear'를 출력하세요.
     반드시 한 단어만 출력하세요: 'booker', 'info', 또는 'unclear'."""),
    ("user", "{request}")
])

In [7]:
if llm:
    coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()

llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x12ab5e8d0>, default_metadata=(), model_kwargs={})

3. Delegation 로직: Runnable 브랜치로 정의

In [8]:
branches = {
    "booker": RunnablePassthrough.assign(output=lambda x: booking_handler(x['request']['request'])),
    "info": RunnablePassthrough.assign(output=lambda x: info_handler(x['request']['request'])),
    "unclear": RunnablePassthrough.assign(output=lambda x: unclear_handler(x['request']['request'])),
}

delegation_branch = RunnableBranch(
    (lambda x: x['decision'].strip() == 'booker', branches["booker"]), # Added .strip()
    (lambda x: x['decision'].strip() == 'info', branches["info"]),     # Added .strip()
    branches["unclear"] # Default branch for 'unclear' or any other output
)

4. 라우터 chain과 delegation branch를 조합하여 single runnable 만들기

In [9]:
# The router chain's output ('decision') is passed along with the original input ('request')
# to the delegation_branch.
coordinator_agent = {
    "decision": coordinator_router_chain,
    "request": RunnablePassthrough()
} | delegation_branch | (lambda x: x['output']) # Extract the final output

In [10]:
def main():
    if not llm:
        print("\nSkipping execution due to LLM initialization failure.")
        return

    print("--- Running with a booking request ---")
    request_a = "Book me a flight to London."
    result_a = coordinator_agent.invoke({"request": request_a})
    print(f"Final Result A: {result_a}")

    print("\n--- Running with an info request ---")
    request_b = "What is the capital of Italy?"
    result_b = coordinator_agent.invoke({"request": request_b})
    print(f"Final Result B: {result_b}")

    print("\n--- Running with an unclear request ---")
    request_c = "Tell me about quantum physics."
    result_c = coordinator_agent.invoke({"request": request_c})
    print(f"Final Result C: {result_c}")

if __name__ == "__main__":
    main()

--- Running with a booking request ---

--- 예약 핸들러에게 위임 중 ---
Final Result A: 예약 핸들러가 요청을 처리했습니다: 'Book me a flight to London.'. 결과: 예약 작업을 시뮬레이션했습니다.

--- Running with an info request ---

--- 정보 핸들러에게 위임 중 ---
Final Result B: 정보 핸들러가 요청을 처리했습니다: 'What is the capital of Italy?'. 결과: 정보 조회를 시뮬레이션했습니다.

--- Running with an unclear request ---

--- 정보 핸들러에게 위임 중 ---
Final Result C: 정보 핸들러가 요청을 처리했습니다: 'Tell me about quantum physics.'. 결과: 정보 조회를 시뮬레이션했습니다.
